In [1]:
import tensorflow as tf
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

#from tensorflow_examples.models.pix2pix import pix2pix
from IPython.display import clear_output

In [2]:
PATH = "data"
PREDICTION_SAVE_PATH = "predictions"
AUTOTUNE = tf.data.experimental.AUTOTUNE
WIDTH, HEIGHT = 512, 512
BATCH_SIZE = 4

In [3]:
def load_data(path):
    images = sorted(glob(f"{path}/images/*.png"))
    masks = sorted(glob(f"{path}/masks/*.png"))
    return images, masks

image_files, mask_files = load_data(PATH)

In [4]:
print(len(image_files), len(mask_files))
DATASET_SIZE = len(image_files)

2667 2667


In [5]:
dataset = tf.data.Dataset.from_tensor_slices((image_files, mask_files))

In [6]:
def normalize(input_image, input_mask):
    input_image = tf.cast(input_image, tf.float32)/ 255.0
    input_mask = tf.cast(input_mask, tf.uint8)
    #input_mask = tf.cast(input_mask, tf.float32)/ 255.0
    #input_mask -= 1
    
    return input_image, input_mask


def parse_image(image_file, mask_file):
    #print(image_file, mask_file)
    img = tf.io.read_file(image_file)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, size=(WIDTH, HEIGHT))
    msk = tf.io.read_file(mask_file)
    msk = tf.image.decode_png(msk, channels=1)
    msk = tf.image.resize(msk, size=(WIDTH, HEIGHT))
    
    img, msk = normalize(img, msk)
    
    return img, msk

In [7]:
dataset = dataset.map(parse_image).batch(BATCH_SIZE)
dataset = dataset.prefetch(buffer_size=AUTOTUNE)

In [8]:
def display(display_list):
    plt.figure(figsize=(WIDTH, HEIGHT))
    print(display_list)
    title = ['input image', 'true mask', 'predicted mask']
    
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()
    
for image, mask in dataset.take(2):
    sample_image, sample_mask = image, mask
    
display([sample_image[1], sample_mask[1]])

[<tf.Tensor: shape=(512, 512, 3), dtype=float32, numpy=
array([[[0.7882353 , 0.78431374, 0.7764706 ],
        [0.7882353 , 0.78431374, 0.7764706 ],
        [0.7873517 , 0.78343016, 0.775587  ],
        ...,
        [0.8509804 , 0.84313726, 0.84705883],
        [0.8509804 , 0.84313726, 0.84705883],
        [0.8509804 , 0.84313726, 0.84705883]],

       [[0.79154414, 0.7876226 , 0.77977943],
        [0.79154414, 0.7876226 , 0.77977943],
        [0.79154414, 0.7876226 , 0.77977943],
        ...,
        [0.8509804 , 0.84313726, 0.84705883],
        [0.8509804 , 0.84313726, 0.84705883],
        [0.8509804 , 0.84313726, 0.84705883]],

       [[0.8       , 0.79607844, 0.7882353 ],
        [0.8       , 0.79607844, 0.7882353 ],
        [0.80036384, 0.7964423 , 0.78859913],
        ...,
        [0.8509804 , 0.84313726, 0.84705883],
        [0.8509804 , 0.84313726, 0.84705883],
        [0.8509804 , 0.84313726, 0.84705883]],

       ...,

       [[0.11103863, 0.10711706, 0.09927392],
        [0.1

In [9]:
print(dataset)

<PrefetchDataset shapes: ((None, 512, 512, 3), (None, 512, 512, 1)), types: (tf.float32, tf.uint8)>


In [10]:
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, Lambda, GlobalAveragePooling2D, concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D, Dropout, MaxPooling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

In [15]:
def get_model():
    in1 = tf.keras.layers.Input(shape=(HEIGHT, WIDTH, 3 ))

    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(in1)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    
    conv41 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(pool4)
    conv41 = Dropout(0.2)(conv41)
    conv41 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv41)
    
    up0 = concatenate([UpSampling2D((2, 2))(conv41), conv4], axis=-1)
    conv42 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up0)
    conv42 = Dropout(0.2)(conv42)
    conv42 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv42)

    up1 = concatenate([UpSampling2D((2, 2))(conv42), conv3], axis=-1)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up1)
    conv5 = Dropout(0.2)(conv5)
    conv5 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv5)
    
    up2 = concatenate([UpSampling2D((2, 2))(conv5), conv2], axis=-1)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv6 = Dropout(0.2)(conv6)
    conv6 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv6)

    up2 = concatenate([UpSampling2D((2, 2))(conv6), conv1], axis=-1)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(up2)
    conv7 = Dropout(0.2)(conv7)
    conv7 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(conv7)
    segmentation = Conv2D(1, (1, 1), activation='sigmoid', name='seg')(conv7)

    model = Model(inputs=[in1], outputs=[segmentation])

    losses = {'seg': 'binary_crossentropy'
            }

    metrics = {'seg': ['acc']
                }
    #adam = tf.keras.optimizers.Adam(learning_rate=0.001)
    #model.compile(optimizer="adam", loss = losses, metrics=metrics)
    model.compile(optimizer='adam'
                  , loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
                  , metrics=metrics)
    return model

In [16]:
model = get_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 512, 512, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
dropout_9 (Dropout)             (None, 512, 512, 32) 0           conv2d_18[0][0]                  
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 512, 512, 32) 9248        dropout_9[0][0]                  
____________________________________________________________________________________________

In [17]:
def save_prediction(predicted, epoch):
    pred_img = tf.keras.preprocessing.image.array_to_img(predicted)
    pred_img.save(f"{PREDICTION_SAVE_PATH}/epoch-{epoch}.png")

def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

def show_predictions(dataset=None, num=2, is_save=False, epoch=0):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      created_mask = create_mask(pred_mask)
      display([image[0], mask[0], created_mask])
      if is_save:
        save_prediction(created_mask, epoch)
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])
    
#show_predictions(dataset, 2, True) 

In [18]:
train_size = int(0.8 * DATASET_SIZE)
val_size = int(0.1 * DATASET_SIZE)
test_size = int(0.1 * DATASET_SIZE)

print(train_size, val_size, test_size)

train_ds = dataset.take(train_size)
test_ds = dataset.skip(train_size)
validation_ds = test_ds.skip(test_size)
test_ds = test_ds.take(test_size)

print(train_ds, test_ds, validation_ds)

2133 266 266
<TakeDataset shapes: ((None, 512, 512, 3), (None, 512, 512, 1)), types: (tf.float32, tf.uint8)> <TakeDataset shapes: ((None, 512, 512, 3), (None, 512, 512, 1)), types: (tf.float32, tf.uint8)> <SkipDataset shapes: ((None, 512, 512, 3), (None, 512, 512, 1)), types: (tf.float32, tf.uint8)>


In [19]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    epoch_num = epoch+1
    show_predictions(dataset, 2, True, epoch_num)
    print ('\nSample Prediction after epoch {}\n'.format(epoch_num))

In [20]:
print(train_ds)
print(test_ds)

<TakeDataset shapes: ((None, 512, 512, 3), (None, 512, 512, 1)), types: (tf.float32, tf.uint8)>
<TakeDataset shapes: ((None, 512, 512, 3), (None, 512, 512, 1)), types: (tf.float32, tf.uint8)>


In [21]:
print(model.input)
print(model.output)

KerasTensor(type_spec=TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 512, 512, 1), dtype=tf.float32, name=None), name='seg/Sigmoid:0', description="created by layer 'seg'")


In [ ]:
EPOCHS = 1
VAL_SUBSPLITS = 5
VALIDATION_STEPS = test_size//BATCH_SIZE//VAL_SUBSPLITS
STEPS_PER_EPOCH = train_size//BATCH_SIZE

#model_history = model.fit(train_ds, epochs=EPOCHS,
#                          steps_per_epoch=STEPS_PER_EPOCH,
#                          validation_steps=VALIDATION_STEPS,
#                          validation_data=test_ds,
#                          callbacks=[DisplayCallback()])

model_history = model.fit(train_ds, epochs=EPOCHS,
                          validation_data=test_ds,
                          shuffle=True,
                          callbacks=[DisplayCallback()])

/Users/chaturawijetunga/work/tools/miniforge3/envs/tfp39/lib/python3.9/site-packages/tensorflow/python/keras/backend.py:4929: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


642/667 [===========================>..] - ETA: 34s - loss: nan - acc: 0.6812